## Loading the dataset

In [1]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import zipfile
zip_ref = zipfile.ZipFile('/content/drive/My Drive/dataset.zip', 'r')
zip_ref.extractall()
zip_ref.close()

## Importing Libraries

In [3]:
import numpy as np
import re
import pickle 
import nltk
from nltk.corpus import stopwords
from sklearn.datasets import load_files
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Unpickling the dataset

In [0]:
with open('dataset/X.pickle','rb') as f:
    X = pickle.load(f)
    
with open('dataset/y.pickle','rb') as f:
    y = pickle.load(f)

## Creating the Corpus

In [0]:
corpus = []
for i in range(len(X)):
    review = re.sub(r'\W', ' ', str(X[i]))
    review = review.lower()
    review = re.sub(r'^br$', ' ', review)
    review = re.sub(r'\s+br\s+',' ',review)
    review = re.sub(r'\s+[a-z]\s+', ' ',review)
    review = re.sub(r'^b\s+', '', review)
    review = re.sub(r'\s+', ' ', review)
    corpus.append(review) 

## Creating the BOW Model

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=2000,min_df = 3,max_df = 0.6,stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(corpus).toarray()

## Convert BOW to TF-TDF model

In [0]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer()
X = transformer.fit_transform(X).toarray()

## Creating the Tf-Idf model directly

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features = 2000, min_df = 3, max_df = 0.6, stop_words = stopwords.words('english'))
X = vectorizer.fit_transform(corpus).toarray()

## Splitting the Data

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.20, random_state = 0,shuffle=True)

## Training the Models

In [10]:
from sklearn.svm import SVC
svm = SVC(C=1000,gamma=0.001)
svm.fit(X_train,Y_train)

SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [11]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(C=6,max_iter = 100000)
classifier.fit(X_train,Y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=6, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100000,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [12]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1000)
rf.fit(X_train,Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

## Saving the Model

In [0]:
!mkdir models
with open('models/vectorizer.pickle','wb') as f:
    pickle.dump(vectorizer,f)

with open('models/svm.pickle','wb') as f:
    pickle.dump(svm,f)
    
with open('models/classifier.pickle','wb') as f:
    pickle.dump(classifier,f)
    
with open('models/rf.pickle','wb') as f:
    pickle.dump(rf,f)

## Testing the Model

In [0]:
from sklearn.metrics import classification_report

In [15]:
## SVM
y_pred = svm.predict(X_test)
print('------------------------------ SVM --------------------------------\n')
print(classification_report(Y_test,y_pred))
print('-------------------------------- XX -------------------------------\n')

## Logistic Regression
y_pred = classifier.predict(X_test)
print('----------------------- Logistic Regression -----------------------\n')
print(classification_report(Y_test,y_pred))
print('-------------------------------- XX -------------------------------\n')

## Random Forest
y_pred = rf.predict(X_test)
print('--------------------------- Random Forest -------------------------\n')
print(classification_report(Y_test,y_pred))
print('-------------------------------- XX -------------------------------\n')

------------------------------ SVM --------------------------------

              precision    recall  f1-score   support

           0       0.88      0.81      0.84       208
           1       0.81      0.89      0.85       192

    accuracy                           0.84       400
   macro avg       0.85      0.85      0.84       400
weighted avg       0.85      0.84      0.84       400

-------------------------------- XX -------------------------------

----------------------- Logistic Regression -----------------------

              precision    recall  f1-score   support

           0       0.90      0.85      0.87       208
           1       0.85      0.90      0.87       192

    accuracy                           0.87       400
   macro avg       0.87      0.87      0.87       400
weighted avg       0.87      0.87      0.87       400

-------------------------------- XX -------------------------------

--------------------------- Random Forest -------------------------

 